<a href="https://colab.research.google.com/github/sathwikmethari/diffusers_ex_collab/blob/main/Diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
ddpm=DDPMPipeline.from_pretrained("google/ddpm-cat-256").to("cuda")
image=ddpm(num_inference_steps=100).images[0]
image

In [ ]:
ddpm_scheduler=DDPMScheduler.from_pretrained("google/ddpm-cat-256")
unet=UNet2DModel.from_pretrained("google/ddpm-cat-256").to(device)

could use .config to change the default values

In [ ]:
ddpm_scheduler.config

In [ ]:
ddpm_scheduler.set_timesteps(50)# 50 time steps between 0 and 1000
ddpm_scheduler.timesteps

In [ ]:
unet.config

In [ ]:
sample_size=unet.config.sample_size
noise = torch.randn((1, 3, sample_size, sample_size)).to("cuda") #creating 1 batch, 3 channel, sample_size*sample_size noise image
print(sample_size)

Meaning of .prev_sample:
In the reverse process, at each time step
t, the model predicts a denoised version of the noisy sample, and then computes the sample at the previous time step
𝑥
𝑡
−
𝑥
𝑡−1
​
 . That is:

𝑥
𝑡
−
1
=
some function of
𝑥
𝑡
 and predicted noise

So in many DDPM implementations, .prev_sample refers to the model’s estimate of the sample at the previous timestep.

Usage:
.prev_sample is commonly returned or stored during inference.

It’s used to track the progression of the denoising chain from

Some implementations (like Hugging Face’s Diffusers library) explicitly name this in their output objects.

In [ ]:
input=noise
for time in ddpm_scheduler.timesteps:
  with torch.inference_mode():
    output_residual=unet(input,time).sample
  input=ddpm_scheduler.step(output_residual,time,input).prev_sample

In [ ]:
from PIL import Image
import numpy as np

image = (input / 2 + 0.5).clamp(0, 1)
image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
image = Image.fromarray((image * 255).round().astype("uint8"))
image